In [3]:
import os
import math
import time
import cv2
import torch

# Initialize YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
inputPath = os.getcwd() + "/test_images/"
outputPath = os.getcwd() + "/output_images/"
os.makedirs(outputPath, exist_ok=True)  # Ensure output directory exists
defaultGreen = 20
defaultYellow = 5
defaultMinimum = 10
defaultMaximum = 60

# Average times for vehicles to pass the intersection
carTime = 2
busTime = 2.5
truckTime = 2.5
motorcycleTime = 1.5
bicycleTime = 1
noOfLanes = 2
#signals = []

# Define vehicle classes to detect
allowed_classes = {"car", "bus", "motorcycle", "truck", "bicycle"}

def detectVehicles(filename):
    """Detect specified vehicle types in the image, save output, and return vehicle counts."""
    vehicle_counts = {vehicle: 0 for vehicle in allowed_classes}
    img = cv2.imread(inputPath + filename, cv2.IMREAD_COLOR)
    results = model(inputPath + filename)
    
    # Extract vehicle details and count only allowed types
    result_labels = results.pandas().xyxy[0]
    for _, vehicle in result_labels.iterrows():
        label = vehicle['name']
        if label in allowed_classes:
            vehicle_counts[label] += 1

            # Draw bounding boxes only for specified vehicles
            x_min, y_min, x_max, y_max = int(vehicle['xmin']), int(vehicle['ymin']), int(vehicle['xmax']), int(vehicle['ymax'])
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(img, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Save the image with detected objects to the output folder
    output_file = os.path.join(outputPath, filename)
    cv2.imwrite(output_file, img)
    
    return vehicle_counts

def calculateGreenTime(vehicle_counts):
    """Calculate green time based on vehicle counts."""
    noOfCars = vehicle_counts.get("car", 0)
    noOfMotorcycle = vehicle_counts.get("motorcycle", 0)
    noOfBicycle = vehicle_counts.get("bicycle", 0)
    noOfBuses = vehicle_counts.get("bus", 0)
    noOfTrucks = vehicle_counts.get("truck", 0)

    # Calculate green signal time
    greenTime = math.ceil(((noOfCars * carTime) + (noOfBicycle * bicycleTime) +
                           (noOfBuses * busTime) + (noOfTrucks * truckTime) +
                           (noOfMotorcycle * motorcycleTime)) / (noOfLanes + 1))
    return max(min(greenTime, defaultMaximum), defaultMinimum)

def calculateRedTime(current_lane_green, all_green_times):
    """Calculate red time for a lane based on the green times of other lanes."""
    return sum(all_green_times) - current_lane_green + (defaultYellow * (len(all_green_times) - 1))

def processLanes():
    """Process each lane and calculate signal timings."""
    image_files = sorted([f for f in os.listdir(inputPath) if f.endswith((".jpg", ".jpeg"))])
    signal_times = []

    all_green_times = []  # To store green times for each lane

    for i, image_file in enumerate(image_files):
        print(f"\nProcessing Lane {i+1} Detection...")
        print("Starting YOLOv5 detection...")
        
        # Step 1: Vehicle detection
        vehicle_counts = detectVehicles(image_file)
        print(f"Detected vehicles: {vehicle_counts}")
        
        # Step 2: Calculate green signal time
        green_time = calculateGreenTime(vehicle_counts)
        all_green_times.append(green_time)
        
        print(f"Calculated green signal time for Lane {i+1}: {green_time} seconds")

        # Display initial signal timings
        red_time = calculateRedTime(green_time, all_green_times)
        print(f"\nInitial Signal Timings for Lane {i+1}:")
        print(f"Red: {red_time} seconds")
        print(f"Green: {green_time} seconds")
        print(f"Yellow: {defaultYellow} seconds\n")
        
        # Store the times for each lane in a tuple format (red, green, yellow)
        signal_times.append((red_time, green_time, defaultYellow))

        # Simulate delay
        time.sleep(1)

    # Final timings for each lane after all lanes are processed
    print("\nFinal Signal Timings for Each Lane After Detection and Calculation:")
    for idx, times in enumerate(signal_times, 1):
        print(f"Lane {idx}: Red = {times[0]}s, Green = {times[1]}s, Yellow = {times[2]}s")

    # Save green signal times 
    with open("signal_times.txt", "w") as f:
        for times in signal_times:
            f.write(f"{times[1]}\n")  # Only write green time 

if __name__ == "__main__":
    # Process lanes and output signal timings
    processLanes()

print("\n")

Using cache found in C:\Users\vaish/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-21 Python-3.12.4 torch-2.5.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\vaish/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):



Processing Lane 1 Detection...
Starting YOLOv5 detection...
Detected vehicles: {'motorcycle': 1, 'bicycle': 0, 'bus': 2, 'truck': 10, 'car': 17}
Calculated green signal time for Lane 1: 22 seconds

Initial Signal Timings for Lane 1:
Red: 0 seconds
Green: 22 seconds
Yellow: 5 seconds


Processing Lane 2 Detection...
Starting YOLOv5 detection...


C:\Users\vaish/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Detected vehicles: {'motorcycle': 4, 'bicycle': 1, 'bus': 1, 'truck': 1, 'car': 4}
Calculated green signal time for Lane 2: 10 seconds

Initial Signal Timings for Lane 2:
Red: 27 seconds
Green: 10 seconds
Yellow: 5 seconds


Processing Lane 3 Detection...
Starting YOLOv5 detection...


C:\Users\vaish/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

Detected vehicles: {'motorcycle': 1, 'bicycle': 1, 'bus': 6, 'truck': 13, 'car': 14}
Calculated green signal time for Lane 6: 26 seconds

Initial Signal Timings for Lane 6:
Red: 118 seconds
Green: 26 seconds
Yellow: 5 seconds


Processing Lane 7 Detection...
Starting YOLOv5 detection...


C:\Users\vaish/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Detected vehicles: {'motorcycle': 4, 'bicycle': 1, 'bus': 6, 'truck': 1, 'car': 10}
Calculated green signal time for Lane 7: 15 seconds

Initial Signal Timings for Lane 7:
Red: 149 seconds
Green: 15 seconds
Yellow: 5 seconds


Processing Lane 8 Detection...
Starting YOLOv5 detection...


C:\Users\vaish/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Detected vehicles: {'motorcycle': 1, 'bicycle': 0, 'bus': 3, 'truck': 7, 'car': 20}
Calculated green signal time for Lane 8: 23 seconds

Initial Signal Timings for Lane 8:
Red: 169 seconds
Green: 23 seconds
Yellow: 5 seconds


Final Signal Timings for Each Lane After Detection and Calculation:
Lane 1: Red = 0s, Green = 22s, Yellow = 5s
Lane 2: Red = 27s, Green = 10s, Yellow = 5s
Lane 3: Red = 42s, Green = 22s, Yellow = 5s
Lane 4: Red = 69s, Green = 23s, Yellow = 5s
Lane 5: Red = 97s, Green = 16s, Yellow = 5s
Lane 6: Red = 118s, Green = 26s, Yellow = 5s
Lane 7: Red = 149s, Green = 15s, Yellow = 5s
Lane 8: Red = 169s, Green = 23s, Yellow = 5s


